In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("Final_Augmented_dataset_Diseases_and_Symptoms.csv")

# Convert binary symptom columns into a list of symptom names
df["symptom_text"] = df.drop(columns=["diseases"]).apply(
    lambda x: [col for col in x.index if x[col] == 1], axis=1
)

# Merge symptoms for each unique disease while ensuring no duplicates
df_grouped = df.groupby("diseases")["symptom_text"].apply(
    lambda x: list(set(sum(x, [])))  # Flatten list and remove duplicates
).reset_index()

# Convert list back to comma-separated string
df_grouped["symptoms"] = df_grouped["symptom_text"].apply(lambda x: ", ".join(sorted(x)))

# Drop intermediate column
df_grouped = df_grouped.drop(columns=["symptom_text"])

# Save the processed dataset
df_grouped.to_csv("processed_symptom_disease.csv", index=False)

print(df_grouped.head())  # Preview the new format

                    diseases                                                                                                                                                                                                  symptoms
0  abdominal aortic aneurysm                                                                                                  arm swelling, back pain, burning abdominal pain, palpitations, sharp abdominal pain, shortness of breath
1           abdominal hernia  ache all over, groin mass, irregular belly button, lower abdominal pain, regurgitation, regurgitation.1, sharp abdominal pain, swollen abdomen, symptoms of the scrotum and testes, upper abdominal pain
2            abscess of nose                                                                                                         coryza, cough, fever, irritable infant, nasal congestion, sinus congestion, sore throat, vomiting
3        abscess of the lung                                                

In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

# Load processed dataset
df = pd.read_csv("processed_symptom_disease.csv")

# Load a pre-trained sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")  # Fast and accurate

# Encode symptom descriptions into dense vectors
df["embedding"] = df["symptoms"].apply(lambda x: model.encode(x))

# Save embeddings (optional, to avoid re-encoding every time)
np.save("symptom_embeddings.npy", np.vstack(df["embedding"].values))
df.to_csv("processed_with_embeddings.csv", index=False)


D:\Work\project\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Work\project\project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\soume\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer m

In [2]:
import faiss
import numpy as np

# Load symptom embeddings
embeddings = np.load("symptom_embeddings.npy")

# Create a FAISS index (L2 distance)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Save FAISS index
faiss.write_index(index, "symptom_faiss.index")

In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss

model = SentenceTransformer("all-MiniLM-L6-v2")  # Fast and accurate

df = pd.read_csv("processed_symptom_disease.csv")

def retrieve_diseases(user_input, model, df, index, top_k=10):
    # Encode user symptoms
    user_embedding = model.encode(user_input).reshape(1, -1)

    # Search FAISS index
    distances, indices = index.search(user_embedding, top_k)

    # Get top-matching diseases
    results = df.iloc[indices[0]][["diseases", "symptoms"]]
    return results

# Load FAISS index
index = faiss.read_index("symptom_faiss.index")

# Example: User input symptoms
user_symptoms = "tiredness, fever, chills, sore throat"
matching_diseases = retrieve_diseases(user_symptoms, model, df, index)

print(matching_diseases)


D:\Work\project\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The variable is a pandas DataFrame.


In [1]:
from langgraph.graph import START, StateGraph, END
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Sequence, Any, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.types import Command
from enum import Enum

from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
from langchain_core.prompts import ChatPromptTemplate

import sys
import os, getpass
from dotenv import load_dotenv
load_dotenv()

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")

model = SentenceTransformer("all-MiniLM-L6-v2")  # Fast and accurate

df = pd.read_csv("processed_symptom_disease.csv")

index = faiss.read_index("symptom_faiss.index")

def retrieve_diseases(user_input, model, df, index, top_k=10):
    # Encode user symptoms
    user_embedding = model.encode(user_input).reshape(1, -1)

    # Search FAISS index
    distances, indices = index.search(user_embedding, top_k)

    # Get top-matching diseases
    results = df.iloc[indices[0]][["diseases", "symptoms"]]
    return results

class Input(TypedDict):
    user_query: str
    symptoms: Optional[List[str]] = None
    final_answer: Optional[str] = None
    diseases: Optional[str] = None

class Source(str, Enum):
    Medical_Query = "Medical Query"
    Generic = "Generic"

base_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def supervisor(input : Input)-> Command[Literal["Analysis", "Help Desk"]]:
    class LLMOutput(TypedDict):
        category: Source
    system_msg = """You are a supervisor routing user query. You have to analyze the provided user query and decide where to route the user query by deciding the category of the query, keeping the following instructions in mind:
                 1. If the user query is requesting for medical assistance or diagnosis related to their symptoms, then assign the category as Medical Query.
                 2. If the user query is of any type other than specified above, then assign the category as Generic.
                 Return as output the category of the user query, which is one of [Medical Query, Generic]
                 """
    messages = [
        ("system", system_msg),
        ("user", input["user_query"])
    ]
    response = base_model.with_structured_output(LLMOutput).invoke(messages)
    print(response)
    if response["category"] == "Medical Query":
        return Command(goto="Analysis", update={
        "user_query":  input["user_query"]
       })
    return Command(goto="Help Desk", update={
        "user_query":  input["user_query"]
    })

def help_desk(input : Input):
    system_msg = """You are an expert medical examiner. You have been provided a generic user query. You have to return a short and brief response explaining to the user that their query can't be answered, and that they should inquire about medical diagnosis instead"""
    messages = [
        ("system", system_msg),
        ("user", input["user_query"])
    ]
    response  = base_model.invoke(ChatPromptTemplate.from_messages(messages).invoke({}))
    return {
        "final_answer": response.content
    }

def analysis(input : Input):
    class LLMOutput(TypedDict):
        symptoms = List[str]
    system_msg = """You are an expert medical examiner.
    You have been provided a user query for medical diagnosis. You have to analyze the query and 
    determine the primary symptoms being experienced by the user.
    Return a list of symptoms being experienced by the user."""
    messages = [
        ("system", system_msg),
        ("user", input["user_query"])
    ]
    response = base_model.with_structured_output(LLMOutput).invoke(messages)
    print(response['symptoms'])
    return{
        "symptoms": response['symptoms']
    }

def diagnosis(input : Input):
    symptoms = ",".join(input["symptoms"])
    diseases = retrieve_diseases(symptoms, model, df, index)
    system_msg = """You are an expert medical examiner.
    You have been provided a user query for medical diagnosis, as well as a list of diseases which might be likely affecting the patient according to their displayed symptoms.
    User query: {query}
    Diseases: {diseases}
    Aanalyze the user query and diseases and provide a detailed possible diagnosis to the user"""
    messages = [
        ("system", system_msg)
    ]
    response  = base_model.invoke(ChatPromptTemplate.from_messages(messages).invoke({
                "query": input["user_query"],
                "diseases": diseases
    }))
    return{
        "diseases": diseases,
        "final_answer": response.content
    }

builder = StateGraph(Input)

builder.add_node("Analysis", analysis)
builder.add_node("Diagnosis", diagnosis)
builder.add_node("Help Desk", help_desk)
builder.add_node("Supervisor", supervisor)

builder.add_edge(START, "Supervisor")
builder.add_edge("Help Desk", END)
builder.add_edge("Analysis", "Diagnosis")
builder.add_edge("Diagnosis", END)

MedBot = builder.compile()

D:\Work\project\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pu = MedBot.invoke(
    {
        "user_query": "I've been feeling really tired lately, and I have a fever with chills. Also, my throat is sore"
    }
)

{'category': 'Medical Query'}
['tiredness', 'fever', 'chills', 'sore throat']


In [5]:
print(pu)

{'user_query': "I've been feeling really tired lately, and I have a fever with chills. Also, my throat is sore", 'symptoms': ['tiredness', 'fever', 'chills', 'sore throat'], 'final_answer': "Based on your symptoms of fatigue, fever with chills, and a sore throat, there are several potential diagnoses to consider. Let's analyze the symptoms in relation to the diseases listed:\n\n1. **Pharyngitis**: This condition is characterized by a sore throat, fever, and can sometimes include cough and hoarseness. Your sore throat and fever align well with this diagnosis.\n\n2. **Strep Throat**: This is a bacterial infection that often presents with a severe sore throat, fever, chills, and general malaise. The presence of chills and a sore throat makes this a strong possibility.\n\n3. **Flu (Influenza)**: The flu typically presents with symptoms such as fever, chills, body aches, and sore throat. Your symptoms of fatigue and chills could indicate the flu, especially if you are experiencing body ache